In [1]:
import open3d as o3d
import numpy as np
import multiprocessing as mp
from multiprocessing import Pool
import copy as cp
import open3d.core as o3c
import matplotlib.pyplot as plt
import pyransac3d as pyrsc
import time
import functions
from scipy.spatial.transform import Rotation
from iteration_utilities import deepflatten
from mpl_toolkits.mplot3d import Axes3D

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#load pcd file, filter, downsample
pcdn = o3d.io.read_point_cloud("final_cropped_ground_align.pcd")
pcdn.estimate_normals()
cl, ind = pcdn.remove_statistical_outlier(nb_neighbors=20,
                                                    std_ratio=0.8)

pcd = pcdn.select_by_index(ind)
pcd = pcd.voxel_down_sample(voxel_size=0.1)
pcd.estimate_normals()
pcd.orient_normals_consistent_tangent_plane(40)
#o3d.visualization.draw_geometries([pcd])

In [4]:

boundaries = functions.detect_boundaries(pcd)



In [6]:
geometries = functions.detect_boundary_patches(boundaries)
o3d.visualization.draw_geometries(geometries+[pcd, boundaries])
   



In [78]:
dimensions = functions.calc_mesh_area_sorted(geometries)
print(dimensions)


[[6.40000000e+01 1.34367073e+02]
 [5.30000000e+01 8.22203293e+01]
 [6.30000000e+01 5.30566329e+01]
 [6.50000000e+01 3.94647418e+01]
 [3.80000000e+01 2.97751760e+01]
 [5.10000000e+01 2.30265756e+01]
 [5.80000000e+01 1.66438714e+01]
 [4.10000000e+01 1.15979364e+01]
 [2.50000000e+01 9.21983918e+00]
 [6.70000000e+01 9.10192739e+00]
 [6.00000000e+00 9.06962939e+00]
 [4.90000000e+01 8.99312101e+00]
 [6.20000000e+01 8.44213139e+00]
 [1.30000000e+01 5.19871867e+00]
 [4.70000000e+01 5.18450272e+00]
 [6.00000000e+01 4.10446575e+00]
 [6.80000000e+01 3.95060609e+00]
 [2.10000000e+01 3.87995394e+00]
 [2.00000000e+00 3.77435251e+00]
 [2.80000000e+01 3.50447226e+00]
 [3.30000000e+01 3.45423659e+00]
 [5.50000000e+01 3.30987222e+00]
 [1.00000000e+01 3.04403565e+00]
 [3.10000000e+01 2.56397496e+00]
 [8.00000000e+00 2.56345055e+00]
 [6.10000000e+01 2.49362430e+00]
 [3.70000000e+01 2.43595569e+00]
 [2.00000000e+01 2.26180034e+00]
 [2.20000000e+01 2.09086703e+00]
 [9.00000000e+00 1.77539398e+00]
 [4.200000

In [57]:
def is_perpendicular(angle, threshold):
    # used to check if a given angle is close to being 90° with certain threshold
    
    if (180 <= angle < 360):
        angle -= 180
        
    if ((90-threshold) <= angle <= (90+threshold)):
        return True
    else:
        return False


In [58]:
def compute_transform(vector1, vector2):
    # compute the translation and rotation needed to transform vector1 to vector2
    
    # Normalize the vectors to unit length
    vector1 = vector1 / np.linalg.norm(vector1)
    vector2 = vector2 / np.linalg.norm(vector2)

    # Compute the rotation matrix
    rotation_matrix = Rotation.align_vectors([vector1], [vector2])[0].as_matrix()

    # Compute the translation vector
    translation_vector = vector2 - np.dot(rotation_matrix, vector1)

    return translation_vector, rotation_matrix

In [59]:
def rotation_matrix_to_axis_angle(matrix):
    # return the angle of rotation around xyz axis from rotation matrix
    r = Rotation.from_matrix(matrix)
    return r.as_euler('xyz', degrees=True)

In [60]:
def invert_normalized_vector(vector):
    # invert the direction of a normalized vector
    inverted_vector = []
    for component in vector:
        inverted_component = -component
        inverted_vector.append(inverted_component)
    return inverted_vector

In [61]:
def check_vector_similar_direction(vec1, vec2):
    # check wether two vectors point in a similar direction
    
    # Normalize the vectors
    normalized_vec1 = np.array(vec1) / np.linalg.norm(vec1)
    normalized_vec2 = np.array(vec2) / np.linalg.norm(vec2)
    
    # Calculate the dot product
    dot_product = np.dot(normalized_vec1, normalized_vec2)
    
    if (dot_product > 0):
        return True
    else:
        return False

In [62]:
def visualize_3d_vectors(vector1, vector2):
    # Create a figure and a 3D axis
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    # normalize vectors
    vector1 = vector1 / np.linalg.norm(vector1)
    vector2 = vector2 / np.linalg.norm(vector2)
    
    if check_vector_similar_direction(vector1, vector2):
        pass
    else:
        pass
        #vector2 = invert_normalized_vector(vector2)
        
    
    # Extract individual coordinates from the vectors
    x1, y1, z1 = vector1
    x2, y2, z2 = vector2
    
    # Plot the vectors as lines
    ax.plot([0, x1], [0, y1], [0, z1], color='r', label='Cam orientation')
    ax.plot([0, x2], [0, y2], [0, z2], color='b', label='Plane normal orientation')
    
    # Set the limits and labels of the plot
    max_x = max(np.max(x1), np.max(x2))
    max_y = max(np.max(y1), np.max(y2))
    max_z = max(np.max(z1), np.max(z2))
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    ax.set_zlim([0, 1])
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    
    # Add a legend
    ax.legend()
    
    # Show the plot
    plt.show()

In [63]:
def angle_between_vectors(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    # Normalize the vectors to unit length
    v1_u = v1 / np.linalg.norm(v1)
    v2_u = v2 / np.linalg.norm(v2)
    angle = np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))
    
    return np.degrees(angle)


In [64]:
def create_arrow_from_vector(vector, location):
    # creates an arrow in form of a triangle mesh
    # places it at a certain location, aligned with the input vector
    
    mesh_normal = vector
    mesh_center = location
    
    #print("Norm average:")
    #print(norm_average)
    
    arrow = o3d.geometry.TriangleMesh.create_cone(0.5,1)
    
    arrow.paint_uniform_color([1,0,0])
    arrow_bb = arrow.get_oriented_bounding_box()
    arrow.compute_triangle_normals()
    arrow.compute_vertex_normals()
    arrow_normal = np.asarray(arrow.vertex_normals)[0]

    translation, rotation = compute_transform(mesh_normal, arrow_normal)  
    
    arrow.rotate(rotation)
    
    arrow.translate(mesh_center, False)
    
    
    arrow_normal = np.asarray(arrow.vertex_normals)[0]
    arrow_center = arrow.get_center()
    same_dir = check_vector_similar_direction(arrow_normal,mesh_normal)
    R_x = np.asarray([[1, 0, 0],
                       [0, -1, 0],
                       [0, 0, -1]])
    
    arrow_cp = o3d.geometry.TriangleMesh.create_cone(0.5,1)
    arrow_cp.paint_uniform_color([1,0,0])
    arrow_cp.compute_triangle_normals()
    arrow_cp.compute_vertex_normals()
    if  (same_dir):
        arrow_cp.rotate(R_x, arrow_center)
        arrow_cp.rotate(rotation)
        arrow_cp.translate(mesh_center, False)
        #return arrow_cp
        return arrow
    else:
        return arrow
    

In [65]:

def count_hits_ray_cast2(scene_answer, cam_normal, dist_thresh, angle_thresh):
    
    
    #scene_answer =  process_data[0]
    #cam_normal   = process_data[1]
    #dist_thresh  = process_data[2]
    #angle_thresh = process_data[3]
    
    
    cast_normals = scene_answer['primitive_normals'].numpy()
    cast_distances = scene_answer['t_hit'].numpy()
    
    dist_lower = dist_thresh[0]
    dist_upper = dist_thresh[1]
    n_valid_hits = 0
    flattened_dist_array = cast_distances.flatten()
    n_pixels = len(flattened_dist_array)
    flattened_normal_array = cast_normals.reshape(n_pixels,3)

    

    
    for i in range(n_pixels):
        
        if (flattened_dist_array[i] >= dist_lower) & (flattened_dist_array[i] <= dist_upper):
            pass
        else:
            continue
        
        point_normal = flattened_normal_array[i]
        angle = angle_between_vectors(point_normal, cam_normal)
        
        if is_perpendicular(angle, angle_thresh):
            continue
        else:
            n_valid_hits += 1
        
            
    #n_valid_hits = np.sum((flattened_dist_array >= dist_lower) & (flattened_dist_array <= dist_upper) & angles)
    
    return n_valid_hits/n_pixels

In [66]:
def rank_all_views2(hole_patches, dist_thresh, angle_thresh):
    
    scene = o3d.t.geometry.RaycastingScene()
    hole_centers = []
    hole_normals = []
    cam_positions = []
    cam_normals = []
    hit_areas = []
    scene_answers = []
    dist_threshes = [dist_thresh]*len(hole_patches)
    angle_threshes = [angle_thresh]*len(hole_patches)
    
    
    for hole in hole_patches:
        scene.add_triangles(o3d.t.geometry.TriangleMesh.from_legacy(hole))
        center = hole.get_center()
        normal = np.asarray(hole.vertex_normals)[0]
        cam_pos = add_vector_to_point(center, normal, 5)
        cam_pos[2] = 0.5
        center[2] = 0.5
        hole_centers.append(center)
        hole_normals.append(normal)
        cam_positions.append(cam_pos)
        cam_normal = (center-cam_pos)
        cam_normals.append(cam_normal)
    
    
   
    for i in range(len(hole_patches)):
        rays = o3d.t.geometry.RaycastingScene.create_rays_pinhole(
                fov_deg=90,
                center=hole_centers[i],
                eye=cam_positions[i],
                up=[0, 0, -1],
                width_px=500,
                height_px=500)
        ans = scene.cast_rays(rays)
        scene_answers.append(ans)
    
    
    
    
    #process_data = zip(scene_answers, cam_normals, dist_threshes, angle_threshes)
    
    with Pool(mp.cpu_count()) as p:
        hit_areas = p.starmap(count_hits_ray_cast2, zip(scene_answers, cam_normals, dist_threshes, angle_threshes))
  
    
    
    return list(zip(hit_areas, cam_positions, cam_normals, scene_answers))



In [67]:
     
ranked_views2 = rank_all_views2(geometries, (2,10), 10 )


#print(ranked_views2)
#print("overall time:", (tok2-tik2))

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>


In [68]:
def sort_xd_list(list, dim, order="ascending"):
    ord = {"ascending" : True,
             "descending" : False}
    list.sort(key=lambda x:x[dim])
    
    if ord[order]:
        return list
    else:
        list.reverse()
        return list
    

In [69]:
#sort views in ascending order according to area covered
#ranked_views2.sort(key=lambda x:x[0])
#ranked_views2.reverse()

ranked_views2 = sort_xd_list(ranked_views2, 0, "ascending")
best_positions = [view[1] for view in ranked_views2]
best_orientations = [view[2] for view in ranked_views2]

print(best_positions)
print(best_orientations)
#best_pos = ranked_views2[-1][1]
#best_orientation = ranked_views2[-1][2]

#print(best_pos, best_orientation)


[array([-31.72848884,  -1.6581284 ,   0.5       ]), array([-39.25294499,  -1.4643561 ,   0.5       ]), array([-43.25040195,  -0.91486952,   0.5       ]), array([ 1.13031533, -1.55115364,  0.5       ]), array([-13.69445787,   4.7836642 ,   0.5       ]), array([-9.74784588,  0.3613949 ,  0.5       ]), array([-13.84137117,   8.59182311,   0.5       ]), array([-0.3975175 ,  6.04761914,  0.5       ]), array([-45.98085311,   4.21808431,   0.5       ]), array([-35.49099991,  -2.18117762,   0.5       ]), array([-17.08093037,   1.92127472,   0.5       ]), array([-2.19223149,  3.38094504,  0.5       ]), array([-14.37673637,   5.78242156,   0.5       ]), array([-2.50566596,  3.91504519,  0.5       ]), array([1.83073908, 3.2410093 , 0.5       ]), array([-45.09687317,  -2.64539161,   0.5       ]), array([-2.42338077, -0.87178342,  0.5       ]), array([-2.71887862, -0.08292731,  0.5       ]), array([-41.85682137,  -2.79064629,   0.5       ]), array([-41.47384822,  -2.82104002,   0.5       ]), array(

In [70]:
#vec1 = np.asarray([-1,0,0])
#vec2 = np.asarray([0,1,2])
#
#_, rot_mat = compute_transform(vec1 ,vec2)
#print(rotation_matrix_to_axis_angle(rot_mat))
#print(angle_between_vectors(vec1, vec2))
#visualize_3d_vectors(vec1 ,vec2)


In [71]:
#for i in range(1,10):
#    print("Areea hit: " , ranked_views2[-i][0])
#    best_pos = ranked_views2[-i][1]
#    best_orientation = ranked_views2[-i][2]
#    best_ans = ranked_views2[-i][3]
#
#    normals = best_ans['primitive_normals'].numpy()
#    #normals = np.abs(best_ans['primitive_normals'].numpy())
#    hits = best_ans['t_hit'].numpy()
#    normals_interp = np.interp(normals, (normals.min(), normals.max()), (0, 1))
#    plt.figure(figsize=(10,6))
#    plt.imshow(normals_interp) 

In [72]:
#best_positions = []
#
#for i in range(1,10):
#    best_pos = ranked_views2[-i][1]
#    best_orientation = ranked_views2[-i][2]
#    best_ans = ranked_views2[-i][3]
#
#    
#    best_positions.append(best_pos)
#    cone = create_arrow_from_vector(best_orientation, best_pos)
#
#    #o3d.visualization.draw_geometries([pcd] + [cone]  + geometries)
#print(best_positions)

In [73]:
np.savetxt("best_pos.csv", best_positions, delimiter="; ")